In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy
import seaborn as sns
%matplotlib inline

In [3]:
df = pd.read_csv('2014BostonMarathon.csv')

In [17]:
df.describe(include = 'O')

,10k,name,25k,gender,bib,ctz,35k,state,30k,5k,half,20k,country,city,40k
count,31984,31984,31984,31984,31984,1244,31984,29408,31984,31984,31984,31984,31984,31983,31984
unique,2766,31915,6570,2,31984,84,9516,68,8090,1491,5651,5375,78,5934,10546
top,-,"Wallace, Sarah",-,M,30317,GBR,-,MA,-,23.93,-,-,USA,Boston,-
freq,50,2,216,17617,1,171,34,7587,40,81,72,51,27233,1034,39
